[![Abrir en Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ManuelEspejo/Machine-Learning-Bases/blob/main/notebooks/03_Aprendizaje-Por-Refuerzo.ipynb) 👈🏻​ **Pulsar para abrir en Colab​**

# ¿Cómo usar estos notebooks?

Si este es el primer notebook que abres en este repositorio, te recomiendo que antes leas el [Manual de uso de estos notebooks](https://github.com/ManuelEspejo/Machine-Learning-Bases/blob/main/docs/manual-notebooks.md) que he creado para que te familiarices con el proyecto y las distintas rutas que puedes seguir, luego puedes volver aquí y continuar.

En este notebook, vamos a profundizar en la **interpretabilidad de los modelos de IA**.

Por otra parte, si aún no has revisado el notebook "[00_Empieza-aquí.ipynb](https://github.com/ManuelEspejo/Machine-Learning-Bases/blob/main/notebooks/00_Empieza-aquí.ipynb)", te sugiero que le eches un vistazo primero para conocer los conceptos básicos. Pero si ya tienes una idea clara de los conceptos básicos y quieres pasar a la práctica, ¡estás en el lugar correcto!

# 04. Interpretabilidad de los Modelos

Cuando entrenamos modelos de Machine Learning, uno de los mayores desafíos no es solo que el modelo funcione bien, sino entender **cómo** y **por qué** toma sus decisiones. Este es el corazón de la interpretabilidad: poder responder a preguntas como *"¿por qué este modelo rechazó mi préstamo?"* o *"¿por qué dice que este tumor es maligno?"*.

## ¿Por qué es importante?

La interpretabilidad de los modelos no es solo un tema técnico, es una cuestión de confianza, responsabilidad y toma de decisiones éticas. Aquí te dejo tres razones principales por las que es crucial:

1. **Confianza en el modelo:** Si no entiendes cómo funciona un modelo, ¿cómo puedes confiar en él? Esto es especialmente importante en áreas críticas como la salud, las finanzas y la justicia, donde las decisiones tienen un impacto directo en la vida de las personas.

2. **Cumplimiento de normativas:** Cada vez más regulaciones exigen que los modelos de IA sean explicables, como el [GDPR en Europa](https://gdpr-info.eu/), que establece el "derecho a explicación" para decisiones automatizadas.

> **Recursos:** Si quieres saber más sobre el GDPR, te recomiendo explorar este [GPT personalizado](https://chatgpt.com/g/g-JvCAAqPCj-gdpr-expert), está muy bien construido.

3. **Impacto social y ético:** Un modelo opaco puede amplificar sesgos y tomar decisiones injustas. Por ejemplo, si un modelo de contratación favorece a un grupo sobre otro sin una justificación clara, estaríamos perpetuando desigualdades.

En resumen, la interpretabilidad no es solo un *"nice to have"*, es una necesidad fundamental en cualquier aplicación de IA que impacte el mundo real.


## ¿Qué vamos a ver?

En este notebook, nos sumergiremos en el fascinante mundo de la interpretabilidad y exploraremos cómo podemos explicar y justificar las decisiones de dos tipos de modelos muy diferentes: un **árbol de decisión** y una **red neuronal**.

Al final de este notebook, habrás aprendido:

- **Qué es la interpretabilidad y por qué importa**, con ejemplos del mundo real.  
- **Cómo entrenar y visualizar un árbol de decisión,** uno de los modelos más intuitivos.  
- **Cómo trabajar con redes neuronales y explorar su interpretabilidad,** utilizando herramientas como SHAP.  
- **La comparación entre ambos enfoques,** destacando los pros y los contras de cada uno.

La meta no es solo entender qué modelo funciona mejor, sino también aprender a tomar decisiones informadas sobre cuándo priorizar la interpretabilidad frente a la precisión.

**¿Listo para desentrañar las decisiones de tus modelos y convertirte en un maestro de la interpretabilidad?**

**¡Vamos allá! 🚀**

# Índice de Contenidos

```
1. Fundamentos de la Interpretabilidad
   1.1. Componentes clave de la interpretabilidad  
       1.1.1. Comprensibilidad  
       1.1.2. Trazabilidad  
       1.1.3. Generalización y robustez  
   1.2. Dimensiones de la interpretabilidad  
       1.2.1. Modelos intrínsecamente interpretables  
       1.2.2. Interpretabilidad post-hoc  
   1.3. Tradeoff: Interpretabilidad vs. Complejidad  
2. Árboles de decisión: Explorando un modelo interpretable (Ejercicio práctico)
   2.1. Carga y exploración del dataset (Breast Cancer Wisconsin)
   2.2. Entrenamiento del árbol de decisión
      2.2.1. Configuración y ajuste de hyperparámetros
      2.2.2. Visualización y análisis del árbol
   2.3. Análisis de la importancia de features
      2.3.1. Identificando las reglas clave
      2.3.2. Interpretación de decisiones específicas
3. Redes neuronales: Desentrañando la "caja negra" (Ejercicio práctico)
   3.1. Construcción y entrenamiento de una red neuronal simple
   3.2. Métodos para interpretar redes neuronales
      3.2.1. Uso de SHAP para analizar la importancia de features
      3.2.2. Visualización de resultados y comparación con el árbol de decisión
4. Comparativa entre modelos: Árboles de decisión vs. Redes neuronales
   4.1. Precisión vs. Interpretabilidad
   4.2. Ventajas y limitaciones de cada enfoque
5. Conclusión
   5.1. Reflexión sobre el tradeoff entre interpretabilidad y precisión
   5.2. El futuro de la interpretabilidad en modelos avanzados
```

In [2]:
import os
import sys

# Configuraciones
# Detectar si estamos en Colab
in_colab = 'google.colab' in str(get_ipython())

if in_colab:
    # Descargar el archivo visualizations.py desde el repositorio de GitHub
    !mkdir -p /content/utils # Creamos una carpeta utils para que coincida con la estructura del repositorio
    !wget -O utils/visualizations.py "https://raw.githubusercontent.com/ManuelEspejo/Machine-Learning-Bases/main/utils/visualizations.py"
    data_dir = '/content/data' # Ruta de los datos
else:
    # Agregar el directorio raíz al path de Python (Para ejecutar en local)
    notebook_dir = os.path.dirname(os.path.abspath('__file__'))
    project_dir = os.path.dirname(notebook_dir)
    sys.path.append(project_dir)
    data_dir = '../data/raw'

In [3]:
# Importar las librerías necesarias
import pandas as pd
from sklearn.datasets import load_breast_cancer

# 1. Fundamentos de la Interpretabilidad

La interpretabilidad no es solo un tema técnico; es lo que conecta a las personas con los modelos. Es como tener un traductor entre el mundo de las matemáticas y el mundo real. Aquí exploraremos los pilares que hacen que un modelo sea interpretable y, lo más importante, entendible.

## 1.1. Componentes clave de la interpretabilidad

### 1.1.1. Comprensibilidad

La facilidad con la que un humano puede entender el funcionamiento interno de un modelo. Por ejemplo, un árbol de decisión es claro porque sigue una lógica "si esto, entonces aquello".

### 1.1.2. Trazabilidad

La capacidad de descomponer una predicción en sus partes para entender qué factores influyeron más. Es como seguir las huellas en la escena de un crímen para reconstruir lo que sucedió.

Por ejemplo, en un modelo médico que predice si un tumor es maligno, queremos saber: ¿Fue el tamaño del tumor? ¿Su forma? ¿La edad del paciente? ¿Todas las características?

Sin trazabilidad, usamos modelos ciegamente. No podemos aceptar un diagnóstico médico sin saber en qué se basó. Aplicar un modelo así puede sonar extremo, pero es la realidad en muchos sistemas de IA actuales, se usan modelos que no sabemos cómo funcionan mucho más frecuentemente de lo que podrías imaginar.

### 1.1.3. Generalización y robustez

Una buena interpretabilidad no solo implica entender un modelo, sino también confiar en que su lógica se mantiene consistente ante nuevas situaciones.

El mundo no es estático, por lo que no queremos un modelo que sólo de buenos resultados en lo que ya ha visto, sino que sea capaz de aplicar su lógica a nuevas situaciones de manera efectiva y sin errores o fallos críticos.

Un modelo robusto debe ser consistente: Si sus decisiones se vuelven erráticas cuando los datos cambian ligeramente, la interpretabilidad pierde su valor.

En definitiva, queremos un modelo que sea capaz de salir de su "zona de confort" y adaptarse a lo que le venga.

## 1.2. Dimensiones de la interpretabilidad

### 1.2.1. Modelo intrínsecamente interpretables

Algunos modelos, como los árboles de decisión y las regresiones lineales, son interpretables por diseño. Su estructura lógica y directa hace que puedas explicarlos como si estuvieras contando una historia.

Te pongo un ejemplo: "Si el paciente tiene más de 50 años y su presión arterial es alta, hay un 80% de probabilidades de que tenga riesgo cardiovascular." Como ves, dos factores que influyen directamente en la variable a predecir.

El problema es que estos modelos a menudo no capturan toda la complejidad de los datos. Son buenos para hacerse una idea de la situación, pero pueden pasar por alto cosas importantes.

### 1.2.2. Interpretabilidad post-hoc

Aquí entran en juego herramientas como `SHAP` o `LIME`. Estas nos sirven para inspeccionar modelos más complejos y entender qué está pasando dentro. Por ejemplo, SHAP te dirá cuánto influyó cada feature en una predicción específica.

Esto sería como tener un panel de control para una máquina compleja. No entiendes todo el sistema, pero las herramientas te permiten ver qué está fallando o funcionando bien.


## 1.3. Tradeoff: Interpretabilidad vs. Complejidad

La vida es un constante balance entre simplicidad y detalle, y lo mismo ocurre con los modelos de Machine Learning. A menudo, enfrentamos el dilema de elegir entre un modelo simple y entendible o uno complejo y más potente y preciso.

Y, como habrás imaginado, no hay una respuesta o una regla que podamos aplicar a todos los casos: dependerá de la situación.

La clave está en decidir qué es más importante según el contexto. En medicina o justicia, probablemente prefieras un modelo más interpretable, incluso si pierdes algo de precisión. En cambio, en aplicaciones como recomendaciones de películas, podrías priorizar la precisión, porque no pasa nada si el modelo falla, de hecho, incluso puede ser divertido y enseñarte cosas que no sabías que te gustaban.

En definitiva, la interpretabilidad no es una meta en sí misma, sino un medio para llegar a decisiones más informadas y conscientes.

# 2. Árboles de decisión: Explorando un modelo interpretable (Ejercicio práctico)

Los árboles de decisión son un excelente punto de partida para entender la interpretabilidad en Machine Learning. Son simples, visuales y fáciles de explicar: como un diagrama de flujo que guía cada decisión basándose en reglas claras. Si quieres explicar cómo tu modelo llegó a una conclusión, un árbol de decisión te lo muestra de forma directa.

Estos árboles nos permiten:
1. Seguir cada decisión paso a paso.
2. Justificar las decisiones de manera clara y sencilla.
3. Son una base ideal para comparar con otros modelos más complejos, como las redes neuronales.

## 2.1. Carga y exploración del dataset (Breast Cancer Wisconsin)

Para comenzar, trabajaremos con el [Breast Cancer Wisconsin Dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html), un conjunto de datos clásico en Machine Learning. Este dataset incluye mediciones médicas de tumores y clasifica si son malignos o benignos.

> **Nota:** Este dataset está disponible en la librería `scikit-learn`, una de las más populares para Machine Learning en Python. Además de este, hay muchos otros más disponibles en [`sklearn.datasets`](https://scikit-learn.org/stable/api/sklearn.datasets.html).

Primero, cargaremos y exploraremos los datos:


In [5]:
# Cargar el dataset
data = load_breast_cancer()
df = pd.DataFrame(data.data, columns=data.feature_names)

# Añadir la columna objetivo (maligno o benigno)
df['target'] = data.target

# Ver las primeras filas del dataset
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


Como podemos apreciar, este dataset contiene los siguientes datos:

- **features (Características):** Contiene 30 características numéricas que describen diferentes aspectos de los tumores, como el tamaño, la forma y la textura. Estas características se dividen en tres categorías principales:
  - *Promedios de medición*, como `mean radius` y `mean texture`.
  - *Errores estándar*, como `radius error` y `texture error`.
  - *Peores valores registrados*, como `worst radius` y `worst texture`.

Esta organización permite capturar tanto una descripción general del tumor como sus posibles irregularidades. Por ejemplo, un error alto en `concavity error` podría indicar bordes irregulares, una señal de malignidad.

- **target (Objetivo):** La columna objetivo indica si el tumor es maligno (`0`) o benigno (`1`).


Ahora, vamos a explorar rápidamente las características principales del dataset.

In [11]:
print(f"Cantidad de instancias: {df.shape[0]}")
print(f"Cantidad de features: {df.shape[1] - 1}")  # Restamos 1 porque 'target' no es feature
print("*"*100, "\n")
print(f"Conteo de las clases:\n{df['target'].value_counts()}") # Conteo de las clases
print("*"*100, "\n")
print(f"Datos faltantes:\n{df.isnull().sum()}")
print("*"*100, "\n")
print(f"Resumen estadístico:\n{df.describe()}") # Resumen estadístico

Cantidad de instancias: 569
Cantidad de features: 30
**************************************************************************************************** 

Conteo de las clases:
target
1    357
0    212
Name: count, dtype: int64
**************************************************************************************************** 

Datos faltantes:
mean radius                0
mean texture               0
mean perimeter             0
mean area                  0
mean smoothness            0
mean compactness           0
mean concavity             0
mean concave points        0
mean symmetry              0
mean fractal dimension     0
radius error               0
texture error              0
perimeter error            0
area error                 0
smoothness error           0
compactness error          0
concavity error            0
concave points error       0
symmetry error             0
fractal dimension error    0
worst radius               0
worst texture              0
worst perimet

Algunas cosas interesantes que podemos ver a simple vista:
- **Equilibrio de clases**: Hay más casos benignos que malignos, pero el desequilibrio no es extremo. Esto es bueno porque significa que el modelo no tendrá un sesgo inicial fuerte hacia una clase.
- **Tamaños de los tumores**: El mean radius varía entre ~6.98 y ~28.1. Esto sugiere que tenemos una gran variedad de tamaños de tumores.
- **Smoothness**: `mean smoothness` tiene un rango mucho más estrecho, lo que indica que los tumores tienden a ser consistentes en esta característica.
- **Valores extremos**: Las columnas `worst radius`, `worst texture` y `worst area` tienen valores más altos que sus contrapartes promedio, lo cual es lógico, ya que capturan los casos más extremos.
- **Datos faltantes**: No tenemos datos faltantes. Esto está genial. Tener un dataset limpio desde el principio facilita el análisis y la modelización. Esto no siempre ocurre en el mundo real, así que estamos en un buen punto de partida.

> **Reflexión:** Los valores promedio y "peores" nos dan un contexto interesante. Por ejemplo, un tumor con un `worst radius` significativamente mayor que su `mean radius` podría ser una señal de crecimiento irregular.


# Recursos


**Datasets**

- [Datasets disponibles en scikit-learn](https://scikit-learn.org/stable/api/sklearn.datasets.html)
- [Breast Cancer Wisconsin Dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html)

**Otros recursos**

- [GDPR](https://gdpr-info.eu/)
- [GPT personalizado sobre GDPR](https://chatgpt.com/g/g-JvCAAqPCj-gdpr-expert)